In [1]:
from slam import fast_slam
from utils.rover import Rover, Pose
from control import pose_control
import numpy as np
import matplotlib.pyplot as plt
import math
from utils.pose import Pose

FastSLAM algorithm implementation is based on particle filters and belongs to the family of probabilistic SLAM approaches. It is used with feature-based maps (see gif above) or with occupancy grid maps.

As it is shown, the particle filter differs from EKF by representing the robot's estimation through a set of particles. Each single particle has an independent belief, as it holds the pose $(x, y, \theta)$ and an array of landmark locations $[(x_1, y_1), (x_2, y_2), ... (x_n, y_n)]$ for n landmarks.

- The blue line is the true trajectory
- The red line is the estimated trajectory
- The red dots represent the distribution of particles
- The black line represent dead reckoning tracjectory
- The blue x is the observed and estimated landmarks
- The black x is the true landmark

I.e. Each particle maintains a deterministic pose and n-EKFs for each landmark and update it with each measurement.

In [ ]:
rov = Rover(0.0,0.0,0.0) # start it at the origin. 

waypoint_range = 2.0
# Waypoint landmarks [x, y, yaw]
waypoint_loc = np.array([[2.0, 2.0, 0.0],
                        [5.0, 3.0, 0.0],
                        [-7.0, 19.0, 0.0],
                        [4.0, 17.0, 0.0]])

# load target waypoints 
pose_goal = []
for waypoint in waypoint_loc:
    pose_goal.append(Pose(waypoint[0], waypoint[1], waypoint[2]))

rov.load_waypoints(pose_goal)
rov.set_target_range(waypoint_range)

sensor_range = 5.0

# consider 5 obstacles/landmarks 
# RFID positions [x, y, yaw]
RFID = np.array([[10.0, -2.0, 0.0],
                [15.0, 10.0, 0.0],
                [3.0, 15.0, 0.0],
                [-5.0, 20.0, 0.0],
                [-5.0, 5.0, 0.0]])

RFID = np.append(RFID, waypoint_loc)
N_LM = RFID.shape[0]


# State Vector [x y yaw v]'
xEst = np.zeros((STATE_SIZE, 1))  # SLAM estimation
xTrue = np.zeros((FastSlam.STATE_SIZE, 1))
xDR = np.zeros((FastSlam.STATE_SIZE, 1))  # Dead reckoning

xTrue[2] = np.deg2rad(45)
xDR[2] = np.deg2rad(45)

# history initial values
hxEst = xEst
hxTrue = xTrue
hxDR = xTrue

## Setting up simulation.
T = 0.0

pose_control.dt = FastSlam.DT = 0.01

particles = [Particle(N_LM) for i in range(N_PARTICLE)]

while not rov.path_complete():
    v = 10
    yaw_rate = 10
    
    while not rov.goal_reached():
        T += FastSlam.DT
        
        v, yaw_rate = pose_control.move_to_pose_step(rov, rov.target())
        
        if abs(v)>10:
            v = math.copysign(10, v)
        if abs(yawrate)>100:
            v = math.copysign(100, yawrate)
            
        u = np.array([[v, yawrate]]).T
        
        xTrue, z, xDR, ud = fast_slam.observation(xTrue, xDR, u, RFID)

        particles = fast_slam.fast_slam(particles, ud, z)

        xEst = calc_final_state(particles)

        x_state = xEst[0: fast_slam.STATE_SIZE]
        
        rov.x = xEst[0][0]
        rov.y = xEst[1][0]
        rov.theta = xEst[2][0]

        # store data history
        hxEst = np.hstack((hxEst, x_state))
        hxDR = np.hstack((hxDR, xDR))
        hxTrue = np.hstack((hxTrue, xTrue))
        
    print("Time take to reach objective " ,i+1," was ", T, " seconds")